In [102]:

import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import pickle
import collections

def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in nltk.word_tokenize(sent)]

path="data/coqa-dev.json"


with open(path,"r")as f:
    train=json.load(f)
    
question_0=collections.Counter()
question_1=collections.Counter()
qa_list=[]
total=0

for paragraph in tqdm(train["data"]):
    context_text=paragraph["story"]
    for i in range(len(paragraph["questions"])):
        question_dict=paragraph["questions"][i]
        answer_dict=paragraph["answers"][i]
        question_text=question_dict["input_text"]
        question=tokenize(question_text)
        answer_text=answer_dict["input_text"]
        sentence_text=answer_dict["span_text"]
        qa_list.append((question_text,answer_text,sentence_text,context_text))
        total+=1
        
        if len(question)>0:
            question_0[question[0]]+=1
            if len(question)>1:
                question_1[" ".join(question[0:2])]+=1

        


100%|██████████| 500/500 [00:00<00:00, 527.06it/s]


In [105]:
for qa in qa_list[1100:1700]:
    q=qa[0]
    print(q)

Why?
Did I think pirates are cool?
Who might be coming up the stairs?
What will tonight be?
Who went to the farm?
Why?
For what?
How many cows did he see there?
Did they have spots?
What color?
What were they doing?
Where?
How did the spots look to him?
Did he move closer?
What leading female character is flying off?
Who was she with?
Who was she leaving?
Was he okay?
Why not?
What else?
What famous talk show host's wife is joining the show?
Who is her wife?
What other show did she star in?
When is Scandal returning?
At what time?
What previous star was jailed?
Where at?
Whose hands does D.C. lie in once she leaves?
What agency does he run?
Who were they waiting for?
Who else?
Who was two hours late?
Where would the carrier go after leaving them?
How was the weather?
What was the employee's name?
Who was his boss?
What were they doing?
Are trees bountiful in that region?
Was this an easy job?
What would they cut with?
What was approaching?
What time of day was it becoming?
And what had

In [5]:
import json
path="data/coqa-dev.json"
with open(path)as f:
    text=json.load(f)
    
text=json.dumps(text,indent=4)
print(text[0:100000])

{
    "version": "1.0",
    "data": [
        {
            "source": "mctest",
            "id": "3dr23u6we5exclen4th8uq9rb42tel",
            "filename": "mc160.test.41",
            "story": "Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n\"What are you doing, C

In [4]:
import json
path="data/squad-dev-v2.0.json"
with open(path)as f:
    text=json.load(f)
    
text=json.dumps(text,indent=4)
print(text[0:100000])

{
    "version": "v2.0",
    "data": [
        {
            "title": "Normans",
            "paragraphs": [
                {
                    "qas": [
                        {
                            "question": "In what country is Normandy located?",
                            "id": "56ddde6b9a695914005b9628",
                            "answers": [
                                {
                                    "text": "France",
                                    "answer_start": 159
                                },
                                {
                                    "text": "France",
                                    "answer_start": 159
                                },
                                {
                                    "text": "France",
                                    "answer_start": 159
                                },
                                {
                                    "text": "France",
          

In [127]:
from pycorenlp import StanfordCoreNLP
from tqdm import tqdm

import matplotlib.pyplot as plt
import networkx as nx


#corenlpを用いて疑問詞とその周辺を取り出す
#ex. How many icecream did you eat? -? How many icecream
#疑問詞を特定できないもの(interro_listの中のタグがあわられないもの)は除去


class CoreNLP():
    def __init__(self):
        self.nlp=StanfordCoreNLP('http://localhost:9000')
        self.interro_list=["WDT","WP"," WP$","WRB","VB","VBD","VBG","VBN","VBP","VBZ"]
        self.count=-1

    def forward(self,text):#input:(batch,seq_len)
        self.count+=1
        q=self.nlp.annotate(text, properties={'annotators': 'tokenize,ssplit,pos','outputFormat': 'json'})
        text=json.dumps(q,indent=4)
        #print(text)
        pos_list=[]
        sentences=q["sentences"]
        for sentence in sentences:
            for word in sentence["tokens"]:
                pos_list.append(word["pos"])
                

        return pos_list


In [29]:
corenlp=CoreNLP()
pos=corenlp.forward("Who was he? I likes coffee. Where are you? I loved him.")
print(pos)

{
    "sentences": [
        {
            "index": 0,
            "tokens": [
                {
                    "index": 1,
                    "word": "Who",
                    "originalText": "Who",
                    "characterOffsetBegin": 0,
                    "characterOffsetEnd": 3,
                    "pos": "WP",
                    "before": "",
                    "after": " "
                },
                {
                    "index": 2,
                    "word": "was",
                    "originalText": "was",
                    "characterOffsetBegin": 4,
                    "characterOffsetEnd": 7,
                    "pos": "VBD",
                    "before": " ",
                    "after": " "
                },
                {
                    "index": 3,
                    "word": "he",
                    "originalText": "he",
                    "characterOffsetBegin": 8,
                    "characterOffsetEnd": 10,
                    "p

In [132]:
#SQuADのデータ処理
#必要条件:CoreNLP
#Tools/core...で
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

import os
import sys
sys.path.append("../")
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import collections
import copy


#sentenceを受け取り、tokenizeして返す
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in word_tokenize(sent)]

#context_textを文分割して、answer_start~answer_end(char単位)のスパンが含まれる文を返す
#やってることはc2iと多分同じアルゴリズム
def answer_find(context_text,answer_start,answer_end,answer_replace):
    context=sent_tokenize(context_text)
    sent_start_id=-1
    sent_end_id=-1
    split_list=[context_text.find(c) for c in context]
    split_list=[(split_list[i],split_list[i+1]) if i+1<len(split_list) else (split_list[i],len(context_text)) for i in range(len(split_list))]
        
    for i,sent in enumerate(context):
        current_p=split_list[i][0]
        end_p=split_list[i][1]
        if current_p<=answer_start and answer_start<=end_p:
            sent_start_id=i
        if current_p<=answer_end and answer_end<=end_p:
            sent_end_id=i
    if sent_start_id==-1 or sent_end_id==-1:
        sys.exit(-1)
    answer_sent=" ".join(context[sent_start_id:sent_end_id+1])
    #ここで答えを置換する方法。ピリオドが消滅した場合などに危険なので止める。
    """
    if answer_replace:
        context_text=context_text.replace(context[answer_start:answer_end],"<answer_word>")
        answer_sent=sent_tokenize(context_text)[sent_start_id]
    """
    return answer_sent,sent_start_id



def data_process(input_path,src_path,tgt_path,question_modify,train=True,sub=False):
    with open(input_path,"r") as f:
        data=json.load(f)
    contexts=[]
    questions=[]
    answer_starts=[]
    answer_ends=[]
    answer_texts=[]
    answers=[]
    sentences=[]
    ids=[]
    answer_replace=False
    cq_list=[]
    corenlp=CoreNLP()
    for paragraph in tqdm(data["data"]):
        context_text=paragraph["story"].lower()
        question_history=[]
        for i in range(len(paragraph["questions"])):
            question_dict=paragraph["questions"][i]
            answer_dict=paragraph["answers"][i]
            question_text=question_dict["input_text"]
            answer_text=answer_dict["input_text"]
            
            #pos
            pos_list=corenlp.forward(text=question_text)
            flag=False
            for pos in pos_list:
                if "VB" in pos:
                    flag=True
                    break


            span_start=answer_dict["span_start"]
            span_end=answer_dict["span_end"]
            span_text=answer_dict["span_text"]
            turn_id=paragraph["questions"][i]["turn_id"]
            if span_start==-1:
                continue
            sentence,answer_index=answer_find(context_text,span_start,span_end,answer_replace)
            context_text=" ".join(tokenize(context_text))
            question_text=" ".join(tokenize(question_text))
            
            question_history.append((question_text,flag))
            cq_list.append((context_text,question_text,answer_index,copy.deepcopy(question_history),answer_text))
            
            #print(cq_list[-1][3])
    return cq_list
                


#main
version="1.1"
type="normal"
question_modify=True
question_interro=True


cq_list=data_process(input_path="data/coqa-dev-v1.0.json",
            src_path="data/coqa-src-dev-{}.txt".format(type),
            tgt_path="data/coqa-tgt-dev-{}.txt".format(type),
            question_modify=True,
            train=False,
            sub=True
            )


100%|██████████| 500/500 [01:37<00:00,  5.12it/s]


In [128]:
with open("data/coqa-dev-v1.0.json","r") as f:
    data=json.load(f)
contexts=[]
questions=[]
answer_starts=[]
answer_ends=[]
answer_texts=[]
answers=[]
sentences=[]
ids=[]
answer_replace=False
corenlp=CoreNLP()
for paragraph in tqdm(data["data"]):
    context_text=paragraph["story"].lower()
    question_history=[]
    for i in range(len(paragraph["questions"])):
        question_dict=paragraph["questions"][i]
        answer_dict=paragraph["answers"][i]
        question_text=question_dict["input_text"]
        answer_text=answer_dict["input_text"]

        #pos
        pos_list=corenlp.forward(text=question_text)
        flag=False
        for pos in pos_list:
            if "VB" in pos:
                flag=True
                break
        print(flag)

  0%|          | 1/500 [00:00<00:58,  8.48it/s]

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


  1%|          | 3/500 [00:00<01:06,  7.49it/s]

True
True
True
True
True
True
False
True
True
True
True
True
True
False
True
True
True
True
True
True


  1%|          | 4/500 [00:00<01:11,  6.93it/s]

...

  7%|▋         | 33/500 [00:04<01:09,  6.72it/s]

True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True


  7%|▋         | 34/500 [00:05<01:09,  6.69it/s]

False
True
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
False
True
True
False
False
True


KeyboardInterrupt: 

In [155]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize,sent_tokenize
from tqdm import tqdm
from nltk.stem.porter import PorterStemmer

def TF_IDF(corpus_text):

    vectorizer = TfidfVectorizer(min_df=0.0,max_df=0.9)
    X = vectorizer.fit_transform(corpus_text)
    return X.toarray()

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    

def sent_retrieve(paragraph,question,num):


    paragraph=sent_tokenize(paragraph)
    paragraph.append(question)
    stemmer = PorterStemmer()
    paragraph=[" ".join([stemmer.stem(w) for w in sentence.split()]) for sentence in paragraph]
    
    mat=TF_IDF(paragraph)
    cos_sim_list={i:cos_sim(mat[i],mat[-1]) for i in range(len(mat)-1)}
    sorted_list=sorted(cos_sim_list.items(),key=lambda x:-x[1])
    index_list=[q[0] for q in sorted_list[0:num]]
    #print(sorted_list[0:num])
    #print(paragraph[index_list[0]])
    #print()
    return index_list

In [173]:

search_list=cq_list

count=0
sum_count=0
length=1000
for i in tqdm(range(1000)):
    #print(search_list[i][3][-1][1])
    if search_list[i][3][-1][1]==True:
        history_length=-2
        continue
    else:
        history_length=-2
    history=" ".join([his[0] for his in  search_list[i][3][history_length:]])
    if search_list[i][2] in sent_retrieve(search_list[i][0],history,1):
        count+=1
    sum_count+=1
        
print(count/sum_count)

    


    



  0%|          | 0/1000 [00:00<?, ?it/s]

 10%|█         | 102/1000 [00:00<00:00, 960.43it/s]

 16%|█▌        | 162/1000 [00:00<00:01, 773.82it/s]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


 27%|██▋       | 271/1000 [00:00<00:00, 855.25it/s]

 33%|███▎      | 329/1000 [00:00<00:00, 776.25it/s]

 39%|███▉      | 389/1000 [00:00<00:00, 736.05it/s]

 45%|████▍     | 446/1000 [00:00<00:00, 706.06it/s]

 54%|█████▍    | 540/1000 [00:00<00:00, 729.80it/s]

 61%|██████    | 606/1000 [00:00<00:00, 708.14it/s]

 69%|██████▉   | 689/1000 [00:00<00:00, 718.73it/s]

 76%|███████▌  | 758/1000 [00:01<00:00, 713.15it/s]

 86%|████████▌ | 858/1000 [00:01<00:00, 737.69it/s]

 94%|█████████▍| 945/1000 [00:01<00:00, 743.45it/s]

100%|██████████| 1000/1000 [00:01<00:00, 740.12it/s]

0.31343283582089554


In [166]:

search_list=cq_list

count=0
sum_count=0
length=1000
for i in tqdm(range(500,600)):
    #print(search_list[i][3][-1][1])
    if search_list[i][3][-1][1]==True:
        continue
    print(sent_tokenize(search_list[i][0])[search_list[i][2]])
    print(search_list[i][1])
    print(search_list[i][2],sent_retrieve(search_list[i][0],search_list[i][1],3))

    
    



  0%|          | 0/100 [00:00<?, ?it/s]

 52%|█████▏    | 52/100 [00:00<00:00, 482.21it/s]

100%|██████████| 100/100 [00:00<00:00, 454.83it/s]



" i should say yes , " came from stanley .
on who ?
5 [21, 2, 14]
put an advertisement of pills on his back and some other ads .
what else ?
2 [0, 20, 25]
in one hand he held the poster tom had fastened on his back , and he was shaking his other fist wrathfully . "
and what else ?
7 [25, 26, 19]
he gasped as he came up . "
what else ?
9 [0, 20, 25]
he has 7 girl cats and only 1 boy cat .
How many ?
2 [0, 1, 2]
each cat gets 3 treats .
How many ?
6 [0, 1, 2]
sometimes he feeds them special treats because he loves them .
Why ?
5 [0, 1, 2]
he does n't give them food like chips and cake and candy , because those foods are n't good for cats .
Why ?
7 [0, 1, 2]
when he first got snowball she was a kitten .
How about Scruff ?
14 [0, 1, 2]
it is so altogether different that i am almost afraid of it , and yet i 've a sneaking idea it is good .
Where ?
5 [0, 1, 2]
chapter xxxiv arthur remained at the gate while ruth climbed maria 's front steps .
Who ?
0 [0, 1, 2]
the cnn-sponsored team announce

In [94]:
length=100
search_list=cq_list

for i in tqdm(range(length)):
    print(sent_tokenize(search_list[i][0])[search_list[i][2]])
    print(search_list[i][1])
    print(search_list[i][2],sent_retrieve(search_list[i][0],search_list[i][1],1))

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
  8%|▊         | 8/100 [00:00<00:01, 70.90it/s]

once upon a time , in a barn near a farm house , there lived a little white kitten named cotton .
What color was Cotton ?
0 [11]
once upon a time , in a barn near a farm house , there lived a little white kitten named cotton .
Where did she live ?
0 [1]
cotton lived high up in a nice warm place above the barn where all of the farmer 's horses slept .
Did she live alone ?
1 [8]
she shared her hay bed with her mommy and 5 other sisters .
Who did she live with ?
3 [3]
the rest of her sisters were all orange with beautiful white tiger stripes like cotton 's mommy .
What color were her sisters ?
6 [4]
the rest of her sisters were all orange with beautiful white tiger stripes like cotton 's mommy .
Was Cotton happy that she looked different than the rest of her family ?
6 [8]
so one day , when cotton found a can of the old farmer 's orange paint , she used it to paint herself like them .
What did she do to try to make herself the same color as her sisters ?
9 [11]
so one day , when cotton fo

 24%|██▍       | 24/100 [00:00<00:01, 71.29it/s]

16 [17]
asta 's papa read the note .
Did a little boy write the note
24 [29]
what does it say ? "
Who could read the note
22 [19]
she wants to be your friend .
did they write back
26 [29]
on the step , i find the elderly chinese lady , small and slight , holding the hand of a little boy .
Who is at the door ?
1 [5]
in her other hand , she holds a paper carrier bag .
Is she carrying something ?
2 [11]
in her other hand , she holds a paper carrier bag .
What ?
2 [9]
in her other hand , she holds a paper carrier bag .
Do I know her ?
2 [3]
she is the boy 's grandmother , and her daughter bought the house next door last october .
Who is her daughter ?
5 [6]
her daughter , nicole , speaks fluent english .
Where does Nicole live ?
6 [20]
it is not her first visit .
How is she related to the boy ?
4 [5]
i know what is inside the bag -- a thermos with hot soup and a stainless-steel container with rice , vegetables and either chicken , meat or shrimp , sometimes with a kind of pancake .
What is

 41%|████      | 41/100 [00:00<00:00, 73.99it/s]

...

8 [9]
in 2004 , he joined the cast of the long-running " law & order " after jerry orbach 's departure , playing detective joe fontana , a role he reprised on the spinoff " trial by jury . "
What did he have ?
8 [1]
in 2004 , he joined the cast of the long-running " law & order " after jerry orbach 's departure , playing detective joe fontana , a role he reprised on the spinoff " trial by jury . "
Were the characters clothes frumpy ?
8 [11]
in 2004 , he joined the cast of the long-running " law & order " after jerry orbach 's departure , playing detective joe fontana , a role he reprised on the spinoff " trial by jury . "
What were they like ?
8 [0]
fontana was known for flashy clothes and an expensive car , a distinct counterpoint to orbach 's rumpled lennie briscoe .
Was he on the show for five years ?
9 [10]
( cnn ) -- dennis farina , the dapper , mustachioed cop-turned-actor best known for his tough-as-nails work in such tv series as " law & order , " " crime story , " and " miami 

 81%|████████  | 81/100 [00:00<00:00, 83.31it/s]

14 [14]
kendra got to school , ran to her teacher and told him what happened that morning .
When she got to school , who did she tell ?
15 [15]
kendra got to school , ran to her teacher and told him what happened that morning .
Did he frown ?
15 [14]
the teacher smiled and told her not to worry , quinton 's mother had called and he was going to the dentist and would be at school after lunch and that she would see him at the bus stop like normal tomorrow .
Who had called ?
16 [16]
the teacher smiled and told her not to worry , quinton 's mother had called and he was going to the dentist and would be at school after lunch and that she would see him at the bus stop like normal tomorrow .
Where did he go ?
16 [14]
the teacher smiled and told her not to worry , quinton 's mother had called and he was going to the dentist and would be at school after lunch and that she would see him at the bus stop like normal tomorrow .
When would he be back ?
16 [14]
staten island is one of the five boroug

100%|██████████| 100/100 [00:01<00:00, 83.90it/s]

1 [2]
a violent storm was coming , but it sounded like his small town would n't be hit too hard .
What was his profession ?
2 [29]
just then , thunder was all-around them .
Did the house lights go out ?
7 [9]
just then , thunder was all-around them .
What device did they use when the power went out ?
7 [30]
eppes held out his arms and ordered his son to jump .
Who was buried under the roof ?
15 [18]
i 've lost him , " eppes thought .
Who did he need to shield ?
19 [20]
quickly , he hurried to joel to shield him .
What were they hit by ?
20 [32]
r.j. had been raised up into the air by the wind and dropped back to the ground .
Was RJ badly hurt ?
33 [34]
amazingly , r.j. was not hurt badly .
Who was hurt the worst ?
34 [34]
then he thought he saw a shape moving straight toward him .
What guided RJ home ?
27 [28]
the search is occurring in the gaithersburg residence of gary giordano , who is currently being held in an aruban jail , fbi special agent rich wolf told cnn .
Whose house was se

In [137]:
count=sum([len(sent_tokenize(cq[0])) for cq in cq_list])
print(count/len(cq_list))

16.34592068704218


In [9]:
#SQuADのデータ処理
#必要条件:CoreNLP
#Tools/core...で
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

import os
import sys
sys.path.append("../")
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import collections
import random
import nltk

from func.corenlp import CoreNLP

def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in nltk.word_tokenize(sent)]

def answer_find(context_text,answer_start,answer_end,answer_replace):
    context=sent_tokenize(context_text)
    sent_start_id=-1
    sent_end_id=-1
    split_list=[context_text.find(c) for c in context]
    split_list=[(split_list[i],split_list[i+1]) if i+1<len(split_list) else (split_list[i],len(context_text)) for i in range(len(split_list))]
        
    for i,sent in enumerate(context):
        current_p=split_list[i][0]
        end_p=split_list[i][1]
        if current_p<=answer_start and answer_start<=end_p:
            sent_start_id=i
        if current_p<=answer_end and answer_end<=end_p:
            sent_end_id=i
    if sent_start_id==-1 or sent_end_id==-1:
        sys.exit(-1)
    answer_sent=" ".join(context[sent_start_id:sent_end_id+1])
    #ここで答えを置換する方法。ピリオドが消滅した場合などに危険なので止める。
    """
    if answer_replace:
        context_text=context_text.replace(context[answer_start:answer_end],"<answer_word>")
        answer_sent=sent_tokenize(context_text)[sent_start_id]
    """
    return answer_sent,sent_start_id


def data_process(input_path,src_path,tgt_path,question_modify,train=True,complete=True,paragraph=True,history=True):
    with open(input_path,"r") as f:
        data=json.load(f)
    contexts=[]
    questions=[]
    answer_starts=[]
    answer_ends=[]
    answer_texts=[]
    answers=[]
    sentences=[]
    ids=[]
    answer_replace=False
    corenlp=CoreNLP()
    count=0
    pos_interro="what"#擬似的な一つ前の疑問詞
    sq_list=[]
    for topic in tqdm(data["data"]):
        topic=topic["paragraphs"]
        for paragraph in topic:
            context_text=paragraph["context"].lower()
            question_history=[]
            for qas in paragraph["qas"]:
                question_text=qas["question"].lower()
                if len(qas["answers"])==0:
                    continue
                a=qas["answers"][0]
                answer_text=a["text"].lower()
                answer_start=a["answer_start"]
                answer_end=a["answer_start"]+len(a["text"])
                sentence,ans_id=answer_find(context_text,answer_start,answer_end,answer_replace)#contextの中からanswerが含まれる文を見つけ出す
                if len(question_text)<=5:#ゴミデータ(10個程度)は削除
                    continue
                answer_text=" ".join(tokenize(answer_text))
                question_text=" ".join(tokenize(question_text))
                context_text=" ".join(tokenize(context_text))
                questions.append(question_text)
                sq_list.append((context_text,question_text,ans_id))



    print(len(questions),len(sentences))
    return sq_list

#main
version="1.1"
type="interro_history"
question_modify=True
question_interro=True

random.seed(0)

"""
data_process(input_path="data/squad-train-v{}.json".format(version),
            src_path="data/squad-src-train-{}.txt".format(type),
            tgt_path="data/squad-tgt-train-{}.txt".format(type),
            question_modify=True,
            train=True,
            complete=True,
            paragraph=False,
            history=True
            )
"""
sq_list=data_process(input_path="data/squad-dev-v{}.json".format(version),
            src_path="data/squad-src-dev-{}.txt".format(type),
            tgt_path="data/squad-tgt-dev-{}.txt".format(type),
            question_modify=True,
            train=False,
            complete=True,
            paragraph=False,
            history=True
            )



100%|██████████| 48/48 [00:16<00:00,  2.95it/s]

10570 0


In [105]:
length=100
search_list=sq_list

for i in tqdm(range(500,700)):
    #print(search_list[i][0])
    print(sent_tokenize(search_list[i][0])[search_list[i][2]])
    print(search_list[i][1])
    print(search_list[i][2],sent_retrieve(search_list[i][0],search_list[i][1],3))

 22%|██▏       | 43/200 [00:00<00:00, 210.77it/s]

the game 's media day , which was typically held on the tuesday afternoon prior to the game , was moved to the monday evening and re-branded as super bowl opening night .
what was media day called for super bowl 50 ?
0 [0, 1, 2]
the event was held on february 1 , 2016 at sap center in san jose .
where was the new media day event for super bowl 50 held ?
1 [0, 1, 2]
alongside the traditional media availabilities , the event featured an opening ceremony with player introductions on a replica of the golden gate bridge .
what bridge had a model of it at the super bowl opening night ?
2 [0, 2, 1]
the event was held on february 1 , 2016 at sap center in san jose .
what date was super bowl opening night held ?
1 [0, 1, 2]
for the first time , the super bowl 50 host committee and the nfl have openly sought disabled veteran and lesbian , gay , bisexual and transgender-owned businesses in business connect , the super bowl program that provides local companies with contracting opportunities in an

 36%|███▌      | 71/200 [00:00<00:00, 171.74it/s]

0 [0, 1, 2]
in the united states , the game was televised by cbs , as part of a cycle between the three main broadcast television partners of the nfl .
what network aired super bowl 50 ?
0 [3, 1, 0]
the network 's lead broadcast team of jim nantz and phil simms called the contest , with tracy wolfson and evan washburn on the sidelines .
who were the announcers of super bowl 50 ?
1 [3, 0, 1]
the network 's lead broadcast team of jim nantz and phil simms called the contest , with tracy wolfson and evan washburn on the sidelines .
who were the super bowl 50 sideline announcers ?
1 [3, 1, 0]
( an earlier version of eyevision was last used in super bowl xxxv ; for super bowl 50 , the cameras were upgraded to 5k resolution . )
what resolution were the eyevision cameras changed to ?
3 [3, 2, 0]
cbs introduced new features during the telecast , including pylon cameras and microphones along with eyevision 360—an array of 36 cameras along the upper deck that can be used to provide a 360-degree v

 56%|█████▌    | 111/200 [00:00<00:00, 180.56it/s]

 [0, 1]
following a break for late local programming , cbs also aired a special episode of the late late show with james corden .
what followed the late local programming after super bowl 50 ?
1 [0, 1]
cbs set the base rate for a 30-second advertisement at $ 5,000,000 , a record high price for a super bowl ad .
what was the base rate for a 30-second ad during super bowl 50 ?
0 [0, 4, 5]
this would be the final year in a multi-year contract with anheuser-busch inbev that allowed the beer manufacturer to air multiple advertisements during the game at a steep discount .
what beverage company signed a contract allowing them to broadcast ads at a discount ?
3 [3, 4, 0]
it was also the final year that doritos , a longtime sponsor of the game , held its " crash the super bowl " contest that allowed viewers to create their own doritos ads for a chance to have it aired during the game .
what brand sponsored the " crash the super bowl " contest ?
4 [4, 0, 5]
nintendo and the pokémon company also

 80%|███████▉  | 159/200 [00:00<00:00, 193.37it/s]

 [0, 1]
westwood one will carry the game throughout north america , with kevin harlan as play-by-play announcer , boomer esiason and dan fouts as color analysts , and james lofton and mark malone as sideline reporters .
who covered the halftime coverage ?
0 [1, 0]
westwood one will carry the game throughout north america , with kevin harlan as play-by-play announcer , boomer esiason and dan fouts as color analysts , and james lofton and mark malone as sideline reporters .
who was the announcer for westwood one 's super bowl 50 coverage ?
0 [0, 1]
westwood one will carry the game throughout north america , with kevin harlan as play-by-play announcer , boomer esiason and dan fouts as color analysts , and james lofton and mark malone as sideline reporters .
who were the westwood one color analysts ?
0 [0, 1]
westwood one will carry the game throughout north america , with kevin harlan as play-by-play announcer , boomer esiason and dan fouts as color analysts , and james lofton and mark ma

100%|██████████| 200/200 [00:01<00:00, 198.38it/s]

 [3, 2, 1]
on january 7 , 2016 , pepsi confirmed to the associated press that beyoncé , who headlined the super bowl xlvii halftime show and collaborated with coldplay on the single " hymn for the weekend " , would be making an appearance .
on january 7 , 2016 , it was confirmed that which start would join coldplay for the halftime show ?
2 [2, 1, 0]
on january 7 , 2016 , pepsi confirmed to the associated press that beyoncé , who headlined the super bowl xlvii halftime show and collaborated with coldplay on the single " hymn for the weekend " , would be making an appearance .
which single did beyoncé and coldplay collaborate on ?
2 [2, 1, 3]
on january 7 , 2016 , pepsi confirmed to the associated press that beyoncé , who headlined the super bowl xlvii halftime show and collaborated with coldplay on the single " hymn for the weekend " , would be making an appearance .
what other former halftime show performer joined coldplay and beyoncé for the super bowl 50 halftime show ?
2 [3, 2, 0]


In [106]:

search_list=sq_list

count=0
sum_count=0
length=1000
for i in tqdm(range(length)):
    if search_list[i][2] in sent_retrieve(search_list[i][0],search_list[i][1],3):
        count+=1
    sum_count+=1
        
print(count/sum_count)


100%|██████████| 1000/1000 [00:04<00:00, 226.43it/s]

0.949
